# Azure AI Agent service - Bing integration

<img src="agent.jpg" width=800>
<img src="foundry.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
#%pip install azure-ai-projects

In [2]:
import datetime
import os
import sys

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import BingGroundingTool
from dotenv import load_dotenv

In [3]:
load_dotenv("azure.env")

True

In [4]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [5]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 12-May-2025 11:35:04


## 1. Grounding Bing Search

In [6]:
name = "Web search agent"
model = "gpt-4o"

instructions = "You are an AI Agent that can do some web search using Bing"
connection_name = "groundingbingsearch"

### Project client

In [7]:
# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

In [8]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.getenv("PROJECT_CONNECTION_STRING"),
)

In [25]:
bing_connection = project_client.connections.get(connection_name=connection_name)
conn_id = bing_connection.id

# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

In [10]:
# Create agent with the bing tool and process assistant run
agent = project_client.agents.create_agent(
    model=model,
    name=name,
    instructions=instructions,
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"}
)

print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_XVbiMbDKlrhdCi0XmX2LbXlF


In [11]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_Nz9H3pXdPN1saN8o4M6W9cZP


In [12]:
query = "What are the latest models of OpenAI?"

In [13]:
# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=query,
)

print(f"Created message, ID: {message.id}")

Created message, ID: msg_mqGgOdEPYFQEHATcnxFqS4Wx


In [14]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
print(f"Last run step detail: {run_steps_data}")

# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")

Run finished with status: completed
Last run step detail: [{'id': 'step_FGA2CutJiWncw464RuNbFMuo', 'object': 'thread.run.step', 'created_at': 1747049712, 'run_id': 'run_xoY0UmCiJUIxf2LNSKmFXv2U', 'assistant_id': 'asst_XVbiMbDKlrhdCi0XmX2LbXlF', 'thread_id': 'thread_Nz9H3pXdPN1saN8o4M6W9cZP', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1747049713, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_uSgFtH6Mhy800ypusbMvXHel'}}, 'usage': {'prompt_tokens': 1460, 'completion_tokens': 75, 'total_tokens': 1535, 'prompt_token_details': {'cached_tokens': 0}}}, {'id': 'step_B31uZA85MCVNZSZZF4OEpc7S', 'object': 'thread.run.step', 'created_at': 1747049711, 'run_id': 'run_xoY0UmCiJUIxf2LNSKmFXv2U', 'assistant_id': 'asst_XVbiMbDKlrhdCi0XmX2LbXlF', 'thread_id': 'thread_Nz9H3pXdPN1saN8o4M6W9cZP', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'com

In [15]:
print("\033[1;31;34m")
print(messages.data[0].content[0].text.value)


The latest models from OpenAI include o3 and o4-mini. The o4-mini is designed for cost-efficient reasoning, excelling in tasks such as math, coding, and visual analysis, while o3 is a highly advanced reasoning model that handles web browsing, image generation, and visual understanding【3:0†source】【3:11†source】.


In [16]:
query = "What is the current stock value for Microsoft?"

In [17]:
# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=query,
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_5uqzowj9AInuMOtjqkmrsqIP


In [18]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
print(f"Last run step detail: {run_steps_data}")

Run finished with status: completed
Last run step detail: [{'id': 'step_o5c9fYetbJOChHEOsGZvqbgs', 'object': 'thread.run.step', 'created_at': 1747049721, 'run_id': 'run_mc1CzsMbnotVJrZem2CTOLFy', 'assistant_id': 'asst_XVbiMbDKlrhdCi0XmX2LbXlF', 'thread_id': 'thread_Nz9H3pXdPN1saN8o4M6W9cZP', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1747049722, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_A81leUKZZn44Z1c5DJkss50I'}}, 'usage': {'prompt_tokens': 1479, 'completion_tokens': 62, 'total_tokens': 1541, 'prompt_token_details': {'cached_tokens': 0}}}, {'id': 'step_yYaGjXaCU3d7POKkjxyd0noR', 'object': 'thread.run.step', 'created_at': 1747049720, 'run_id': 'run_mc1CzsMbnotVJrZem2CTOLFy', 'assistant_id': 'asst_XVbiMbDKlrhdCi0XmX2LbXlF', 'thread_id': 'thread_Nz9H3pXdPN1saN8o4M6W9cZP', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'com

In [19]:
print("\033[1;31;34m")
messages = project_client.agents.list_messages(thread_id=thread.id)
print(messages.data[0].content[0].text.value)


The most recent available closing price for Microsoft's stock (MSFT) is $438.73 as of May 9, 2025, reflecting a slight increase from previous trading【7:2†source】【7:9†source】. Please note that stock values may have changed since then.


In [20]:
query = "What are the weather informations for Paris? Use emojis and be funny"

In [21]:
# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=query,
)
print(f"Created message, ID: {message.id}")

Created message, ID: msg_EANGwMDl3NVJl1tt4b64cHkn


In [22]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
print(f"Last run step detail: {run_steps_data}")

Run finished with status: completed
Last run step detail: [{'id': 'step_yf7eaODCqFhiqRhOxjqCRhzB', 'object': 'thread.run.step', 'created_at': 1747049728, 'run_id': 'run_COmGoN7zAEJBLwdwTCH9Mh3x', 'assistant_id': 'asst_XVbiMbDKlrhdCi0XmX2LbXlF', 'thread_id': 'thread_Nz9H3pXdPN1saN8o4M6W9cZP', 'type': 'message_creation', 'status': 'completed', 'cancelled_at': None, 'completed_at': 1747049729, 'expires_at': None, 'failed_at': None, 'last_error': None, 'step_details': {'type': 'message_creation', 'message_creation': {'message_id': 'msg_izpJlirswRtNKyQlrMLQTFWL'}}, 'usage': {'prompt_tokens': 1424, 'completion_tokens': 93, 'total_tokens': 1517, 'prompt_token_details': {'cached_tokens': 0}}}, {'id': 'step_6UmehEDjkDjXHYYsGnmosmUT', 'object': 'thread.run.step', 'created_at': 1747049726, 'run_id': 'run_COmGoN7zAEJBLwdwTCH9Mh3x', 'assistant_id': 'asst_XVbiMbDKlrhdCi0XmX2LbXlF', 'thread_id': 'thread_Nz9H3pXdPN1saN8o4M6W9cZP', 'type': 'tool_calls', 'status': 'completed', 'cancelled_at': None, 'com

In [23]:
print("\033[1;31;34m")
messages = project_client.agents.list_messages(thread_id=thread.id)
print(messages.data[0].content[0].text.value)


Paris today is feeling a bit dramatic! 🌥️ It's throwing a cloud parade during the day (12°C to 19°C), with some surprise sprinkles of rain 🌧️ to keep things cool. But don't worry—tonight promises a clear sky for stargazing 🌌! So, pack a raincoat and your good mood, Parisian style 🇫🇷【11:0†source】【11:1†source】!


### Post processing

In [24]:
agents = project_client.agents.list_agents()

for i in range(len(agents.data)):
    if agents.data[i].name == name:
        print(f"Deleting agent {agents.data[i].id}")
        project_client.agents.delete_agent(agents.data[i].id)

Deleting agent asst_XVbiMbDKlrhdCi0XmX2LbXlF
